# SPEED
## speed tests

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split, Subset
from sklearn.preprocessing import LabelEncoder
from IPython.display import clear_output
import os
import time
import random
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from models import *
from data_loader import *
from data_format import *
from datapath import *
from IPython.display import clear_output
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:18230"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device=torch.device("cpu")
print(f"Using device: {device}")
csfont = {'fontname':'Times New Roman'}
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
#


Using device: cuda
Using device: cuda
Using device: cuda


## create dataset

In [8]:
#get varying sizes of the dataset
"""data=loaded(t=-1,filename="X_data_15.npz")
data.applySobel()
data.orientation_augment()
data.speed_augment(4,speeds=3)
data.zoom_augment([10,20,30,40])"""

'data=loaded(t=-1,filename="X_data_15.npz")\ndata.applySobel()\ndata.orientation_augment()\ndata.speed_augment(4,speeds=3)\ndata.zoom_augment([10,20,30,40])'

## experiments

In [2]:
#global parameters
test_data=np.load("/its/home/drs25/RoboSkin/Code/NewRigExperiments/texture-tactip/concatenated_data_handheld.npy").astype(np.uint8)
labels=np.load("/its/home/drs25/RoboSkin/Code/NewRigExperiments/texture-tactip/concatenated_labels_handheld.npy").astype(np.uint8)
test_data_sliced=test_data[:,::4,:,:]
test_data_sliced=test_data_sliced[:,10:10+4,:,:]
crop=[60,180,40,150]
im=test_data_sliced[:,:,crop[2]:crop[3],crop[0]:crop[1]]
new=np.zeros((im.shape[0],im.shape[1],im.shape[2],im.shape[3]))
for i in range(len(test_data_sliced)): #crop all images individually
    for j in range(len(test_data_sliced[0])):
        image=cv2.resize(test_data_sliced[i][j],(int(640*0.4),int(480*0.4)),interpolation=cv2.INTER_AREA)
        image=cv2.cvtColor(image[crop[2]:crop[3],crop[0]:crop[1]], cv2.COLOR_BGR2GRAY)
        # Apply Sobel filter in x-direction
        sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)  # ksize=3 for a 3x3 Sobel kernel

        # Apply Sobel filter in y-direction
        sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)

        # Convert the results back to uint8
        sobel_x = np.uint8(np.absolute(sobel_x))
        sobel_y = np.uint8(np.absolute(sobel_y))

        # Combine the results to get the final edge-detected image
        sobel_combined = cv2.bitwise_or(sobel_x, sobel_y)
        new[i][j]=sobel_combined
print(new.shape)
new=new.reshape(new.shape[0],1,4*new.shape[2],new.shape[3])
print(new.shape)
new=(new-np.mean(new))/(np.max(new)-np.min(new))
test_data_sliced_tensor=torch.tensor(new,dtype=torch.float64).float().to(device)


def calc_noisy(preds,labels):
    names=['Carpet', 'LacedMatt', 'wool', 'Cork', 'Felt', 'LongCarpet', 'cotton', 'Plastic', 'Flat', 'Ffoam', 'Gfoam', 'bubble', 'Efoam', 'jeans', 'Leather']
    the_other_labels=['Cork','FoamF','FoamG','LacedMatt','Leather','LongCarpet','Plastic','ShortCarpet','Bubble','FoamE']
    mapping={0:7,1:3,3:0,5:5,7:6,9:1,10:2,11:8,12:9,14:4}
    classes=torch.argmax(preds,axis=1)
    correct=0
    for i in range(len(classes)):
        if mapping.get(classes[i],-1) >0:
            if mapping[classes[i]]==labels[i]: correct+=1
    return correct/len(classes) *100

num_trials=20
T_SIZE=4
EPOCHS=100

(50, 4, 110, 120)
(50, 1, 440, 120)


### experiment with different zooms

In [10]:
## zoom experiments
acc=np.zeros((num_trials,3)) #train,test,noisy

for i in range(num_trials):
    data=loaded(t=T_SIZE,filename="X_data_15.npz")
    data.applySobel()
    data.zoom_augment([10,20,30,40])
    train_loader,test_loader=org_data(data,(len(data.X),1,T_SIZE*data.X.shape[2],data.X.shape[3]))
    model,history=run(train_loader,0,4,num_epochs = EPOCHS)
    print("Commence calculations...")
    acc[i][0]=calc(model,train_loader)
    acc[i][1]=calc(model,test_loader)
    preds=model(test_data_sliced_tensor)
    acc[i][2]=calc_noisy(preds,labels)
    print("Results:",acc[i])
np.save(path+"/data/20_TRIALS_CNN_ZOOM_AUGMENTATION",acc)

Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(12000, 4, 110, 120) (12000, 1, 440, 120)
Using 4.72 GB
torch.Size([12000, 1, 440, 120])
torch.Size([12000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [240/240], Loss: 649.4830
Epoch [11/100], Step [240/240], Loss: 58.8220
Epoch [21/100], Step [240/240], Loss: 8.1832
Epoch [31/100], Step [240/240], Loss: 1.1230
Epoch [41/100], Step [240/240], Loss: 0.4444
Epoch [51/100], Step [240/240], Loss: 0.2812
Epoch [61/100], Step [240/240], Loss: 0.1829
Epoch [71/100], Step [240/240], Loss: 0.1387
Epoch [81/100], Step [240/240], Loss: 0.1139
Epoch [91/100], Step [240/240], Loss: 0.0922
Finished Training
Commence calculations...
Accuracy: 99.9989583441839 %
Accuracy: 99.99583350693722 %
Results: [0.99998958 0.99995834 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(12000, 4, 110, 120) (12000, 1, 440, 120)
Using 4.72 GB
torch.Size([12000, 1, 440, 120])
torch.Size([12000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [240/240], Loss: 649.7297
Epoch [11/100], Step [240/240], Loss: 69.6712
Epoch [21/100], Step [240/240], Loss: 8.3316
Epoch [31/100], Step [240/240], Loss: 1.5385
Epoch [41/100], Step [240/240], Loss: 0.7060
Epoch [51/100], Step [240/240], Loss: 0.3964
Epoch [61/100], Step [240/240], Loss: 0.1987
Epoch [71/100], Step [240/240], Loss: 0.1486
Epoch [81/100], Step [240/240], Loss: 0.1174
Epoch [91/100], Step [240/240], Loss: 0.0998
Finished Training
Commence calculations...
Accuracy: 99.9989583441839 %
Accuracy: 99.99583350693722 %
Results: [0.99998958 0.99995834 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(12000, 4, 110, 120) (12000, 1, 440, 120)
Using 4.72 GB
torch.Size([12000, 1, 440, 120])
torch.Size([12000, 15])


...

Epoch [1/100], Step [240/240], Loss: 649.7232
Epoch [11/100], Step [240/240], Loss: 54.1726
Epoch [21/100], Step [240/240], Loss: 7.5074
Epoch [31/100], Step [240/240], Loss: 1.4484
Epoch [41/100], Step [240/240], Loss: 0.4229
Epoch [51/100], Step [240/240], Loss: 0.2665
Epoch [61/100], Step [240/240], Loss: 0.1881
Epoch [71/100], Step [240/240], Loss: 0.1438
Epoch [81/100], Step [240/240], Loss: 0.1162
Epoch [91/100], Step [240/240], Loss: 0.0941
Finished Training
Commence calculations...
Accuracy: 99.9989583441839 %
Accuracy: 99.99583350693722 %
Results: [0.99998958 0.99995834 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(12000, 4, 110, 120) (12000, 1, 440, 120)
Using 4.72 GB
torch.Size([12000, 1, 440, 120])
torch.Size([12000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [240/240], Loss: 649.9055
Epoch [11/100], Step [240/240], Loss: 67.1395
Epoch [21/100], Step [240/240], Loss: 8.2876
Epoch [31/100], Step [240/240], Loss: 1.4156
Epoch [41/100], Step [240/240], Loss: 0.5564
Epoch [51/100], Step [240/240], Loss: 0.3391
Epoch [61/100], Step [240/240], Loss: 0.1871
Epoch [71/100], Step [240/240], Loss: 0.1509
Epoch [81/100], Step [240/240], Loss: 0.1167
Epoch [91/100], Step [240/240], Loss: 0.0976
Finished Training
Commence calculations...
Accuracy: 99.9989583441839 %
Accuracy: 99.99583350693722 %
Results: [0.99998958 0.99995834 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(12000, 4, 110, 120) (12000, 1, 440, 120)
Using 4.72 GB
torch.Size([12000, 1, 440, 120])
torch.Size([12000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [240/240], Loss: 650.2483
Epoch [11/100], Step [240/240], Loss: 105.0007
Epoch [21/100], Step [240/240], Loss: 12.7099
Epoch [31/100], Step [240/240], Loss: 2.6175
Epoch [41/100], Step [240/240], Loss: 0.8251
Epoch [51/100], Step [240/240], Loss: 0.3645
Epoch [61/100], Step [240/240], Loss: 0.2476
Epoch [71/100], Step [240/240], Loss: 0.1649
Epoch [81/100], Step [240/240], Loss: 0.1242
Epoch [91/100], Step [240/240], Loss: 0.1019
Finished Training
Commence calculations...
Accuracy: 99.9989583441839 %
Accuracy: 99.99583350693722 %
Results: [0.99998958 0.99995834 0.        ]


### experiment with different speeds

In [11]:
acc=np.zeros((num_trials,3)) #train,test,noisy

for i in range(num_trials):
    data=loaded(t=-1,filename="X_data_15.npz")
    data.applySobel()
    data.speed_augment(t=4)
    train_loader,test_loader=org_data(data,(len(data.X),1,T_SIZE*data.X.shape[2],data.X.shape[3]))
    model,history=run(train_loader,0,4,num_epochs = EPOCHS)
    print("Commence calculations...")
    acc[i][0]=calc(model,train_loader)
    acc[i][1]=calc(model,test_loader)
    preds=model(test_data_sliced_tensor)
    acc[i][2]=calc_noisy(preds,labels)
    print("Results:",acc[i])
np.save(path+"/data/20_TRIALS_CNN_SPEED_AUGMENTATION",acc)

Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 485.3604
Epoch [11/100], Step [180/180], Loss: 25.1672
Epoch [21/100], Step [180/180], Loss: 0.9273
Epoch [31/100], Step [180/180], Loss: 0.3135
Epoch [41/100], Step [180/180], Loss: 0.1763
Epoch [51/100], Step [180/180], Loss: 0.1189
Epoch [61/100], Step [180/180], Loss: 0.0875
Epoch [71/100], Step [180/180], Loss: 0.0688
Epoch [81/100], Step [180/180], Loss: 0.0566
Epoch [91/100], Step [180/180], Loss: 0.0475
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 485.0128
Epoch [11/100], Step [180/180], Loss: 17.4700
Epoch [21/100], Step [180/180], Loss: 0.6779
Epoch [31/100], Step [180/180], Loss: 0.2675
Epoch [41/100], Step [180/180], Loss: 0.1579
Epoch [51/100], Step [180/180], Loss: 0.1093
Epoch [61/100], Step [180/180], Loss: 0.0818
Epoch [71/100], Step [180/180], Loss: 0.0646
Epoch [81/100], Step [180/180], Loss: 0.0536
Epoch [91/100], Step [180/180], Loss: 0.0454
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


...

Epoch [1/100], Step [180/180], Loss: 484.6801
Epoch [11/100], Step [180/180], Loss: 16.3481
Epoch [21/100], Step [180/180], Loss: 0.6829
Epoch [31/100], Step [180/180], Loss: 0.2830
Epoch [41/100], Step [180/180], Loss: 0.1626
Epoch [51/100], Step [180/180], Loss: 0.1134
Epoch [61/100], Step [180/180], Loss: 0.0859
Epoch [71/100], Step [180/180], Loss: 0.0671
Epoch [81/100], Step [180/180], Loss: 0.0548
Epoch [91/100], Step [180/180], Loss: 0.0470
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 481.2628
Epoch [11/100], Step [180/180], Loss: 21.2275
Epoch [21/100], Step [180/180], Loss: 0.9144
Epoch [31/100], Step [180/180], Loss: 0.2971
Epoch [41/100], Step [180/180], Loss: 0.1666
Epoch [51/100], Step [180/180], Loss: 0.1146
Epoch [61/100], Step [180/180], Loss: 0.0853
Epoch [71/100], Step [180/180], Loss: 0.0677
Epoch [81/100], Step [180/180], Loss: 0.0546
Epoch [91/100], Step [180/180], Loss: 0.0462
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 485.0050
Epoch [11/100], Step [180/180], Loss: 19.0047
Epoch [21/100], Step [180/180], Loss: 0.8216
Epoch [31/100], Step [180/180], Loss: 0.3436
Epoch [41/100], Step [180/180], Loss: 0.1634
Epoch [51/100], Step [180/180], Loss: 0.1120
Epoch [61/100], Step [180/180], Loss: 0.0838
Epoch [71/100], Step [180/180], Loss: 0.0663
Epoch [81/100], Step [180/180], Loss: 0.0545
Epoch [91/100], Step [180/180], Loss: 0.0459
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]


### orientation augment

In [12]:
acc=np.zeros((num_trials,3)) #train,test,noisy

for i in range(num_trials):
    data=loaded(t=T_SIZE,filename="X_data_15.npz")
    data.applySobel()
    data.orientation_augment()
    train_loader,test_loader=org_data(data,(len(data.X),1,T_SIZE*data.X.shape[2],data.X.shape[3]))
    model,history=run(train_loader,0,4,num_epochs = EPOCHS)
    print("Commence calculations...")
    acc[i][0]=calc(model,train_loader)
    acc[i][1]=calc(model,test_loader)
    preds=model(test_data_sliced_tensor)
    acc[i][2]=calc_noisy(preds,labels)
    print("Results:",acc[i])
np.save(path+"/data/20_TRIALS_CNN_ORIENTATION_AUGMENTATION",acc)

Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 9000 
Window size: 4 
Image: (110, 120)
Memory needed: 0.44 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 487.4223
Epoch [11/100], Step [180/180], Loss: 131.3247
Epoch [21/100], Step [180/180], Loss: 27.2487
Epoch [31/100], Step [180/180], Loss: 8.5030
Epoch [41/100], Step [180/180], Loss: 2.7641
Epoch [51/100], Step [180/180], Loss: 1.1623
Epoch [61/100], Step [180/180], Loss: 0.2506
Epoch [71/100], Step [180/180], Loss: 0.1479
Epoch [81/100], Step [180/180], Loss: 0.1103
Epoch [91/100], Step [180/180], Loss: 0.0853
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.93889228376202 %
Results: [0.99998611 0.99938892 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 9000 
Window size: 4 
Image: (110, 120)
Memory needed: 0.44 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 486.2532
Epoch [11/100], Step [180/180], Loss: 76.7832
Epoch [21/100], Step [180/180], Loss: 23.8296
Epoch [31/100], Step [180/180], Loss: 7.9138
Epoch [41/100], Step [180/180], Loss: 1.5108
Epoch [51/100], Step [180/180], Loss: 0.5081
Epoch [61/100], Step [180/180], Loss: 0.1975
Epoch [71/100], Step [180/180], Loss: 0.1265
Epoch [81/100], Step [180/180], Loss: 0.1008
Epoch [91/100], Step [180/180], Loss: 0.0795
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 9000 
Window size: 4 
Image: (110, 120)
Memory needed: 0.44 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


...

Epoch [1/100], Step [180/180], Loss: 487.2269
Epoch [11/100], Step [180/180], Loss: 83.9666
Epoch [21/100], Step [180/180], Loss: 21.5326
Epoch [31/100], Step [180/180], Loss: 5.5495
Epoch [41/100], Step [180/180], Loss: 1.6127
Epoch [51/100], Step [180/180], Loss: 0.3744
Epoch [61/100], Step [180/180], Loss: 0.1763
Epoch [71/100], Step [180/180], Loss: 0.1255
Epoch [81/100], Step [180/180], Loss: 0.0968
Epoch [91/100], Step [180/180], Loss: 0.0778
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 9000 
Window size: 4 
Image: (110, 120)
Memory needed: 0.44 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 485.3154
Epoch [11/100], Step [180/180], Loss: 64.0228
Epoch [21/100], Step [180/180], Loss: 20.1430
Epoch [31/100], Step [180/180], Loss: 5.2083
Epoch [41/100], Step [180/180], Loss: 0.7104
Epoch [51/100], Step [180/180], Loss: 0.2334
Epoch [61/100], Step [180/180], Loss: 0.1594
Epoch [71/100], Step [180/180], Loss: 0.1136
Epoch [81/100], Step [180/180], Loss: 0.0893
Epoch [91/100], Step [180/180], Loss: 0.0732
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.93889228376202 %
Results: [0.99998611 0.99938892 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
Dataset size: 9000 
Window size: 4 
Image: (110, 120)
Memory needed: 0.44 GB
LOADED DATASET...
Memory left 23.59 GB
(9000, 4, 110, 120) (9000, 1, 440, 120)
Using 3.54 GB
torch.Size([9000, 1, 440, 120])
torch.Size([9000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [180/180], Loss: 484.9257
Epoch [11/100], Step [180/180], Loss: 66.1417
Epoch [21/100], Step [180/180], Loss: 19.7765
Epoch [31/100], Step [180/180], Loss: 6.5871
Epoch [41/100], Step [180/180], Loss: 1.4695
Epoch [51/100], Step [180/180], Loss: 0.5205
Epoch [61/100], Step [180/180], Loss: 0.1618
Epoch [71/100], Step [180/180], Loss: 0.1187
Epoch [81/100], Step [180/180], Loss: 0.0909
Epoch [91/100], Step [180/180], Loss: 0.0742
Finished Training
Commence calculations...
Accuracy: 99.99861113040096 %
Accuracy: 99.99444475306927 %
Results: [0.99998611 0.99994445 0.        ]


### no augmentation

In [16]:
acc=np.zeros((num_trials,3)) #train,test,noisy

for i in range(num_trials):
    data=loaded(t=T_SIZE,filename="X_data_15.npz")
    data.applySobel()
    train_loader,test_loader=org_data(data,(len(data.X),1,T_SIZE*data.X.shape[2],data.X.shape[3]))
    model,history=run(train_loader,0,4,num_epochs = EPOCHS)
    print("Commence calculations...")
    acc[i][0]=calc(model,train_loader)
    acc[i][1]=calc(model,test_loader)
    preds=model(test_data_sliced_tensor)
    acc[i][2]=calc_noisy(preds,labels)
    print("Results:",acc[i])
np.save(path+"/data/20_TRIALS_CNN_NO_AUGMENTATION",acc)

Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(3000, 4, 110, 120) (3000, 1, 440, 120)
Using 1.18 GB
torch.Size([3000, 1, 440, 120])
torch.Size([3000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [60/60], Loss: 162.3671
Epoch [11/100], Step [60/60], Loss: 72.5590
Epoch [21/100], Step [60/60], Loss: 18.6926
Epoch [31/100], Step [60/60], Loss: 6.5712
Epoch [41/100], Step [60/60], Loss: 2.7608
Epoch [51/100], Step [60/60], Loss: 0.7963
Epoch [61/100], Step [60/60], Loss: 0.2581
Epoch [71/100], Step [60/60], Loss: 0.1674
Epoch [81/100], Step [60/60], Loss: 0.1189
Epoch [91/100], Step [60/60], Loss: 0.0927
Finished Training
Commence calculations...
Accuracy: 99.99583350693722 %
Accuracy: 99.98333611064822 %
Results: [0.99995834 0.99983336 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(3000, 4, 110, 120) (3000, 1, 440, 120)
Using 1.18 GB
torch.Size([3000, 1, 440, 120])
torch.Size([3000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [60/60], Loss: 162.3850
Epoch [11/100], Step [60/60], Loss: 72.8448
Epoch [21/100], Step [60/60], Loss: 18.6390
Epoch [31/100], Step [60/60], Loss: 7.0444
Epoch [41/100], Step [60/60], Loss: 1.9303
Epoch [51/100], Step [60/60], Loss: 0.6517
Epoch [61/100], Step [60/60], Loss: 0.2138
Epoch [71/100], Step [60/60], Loss: 0.1473
Epoch [81/100], Step [60/60], Loss: 0.1084
Epoch [91/100], Step [60/60], Loss: 0.0859
Finished Training
Commence calculations...
Accuracy: 99.99583350693722 %
Accuracy: 99.98333611064822 %
Results: [0.99995834 0.99983336 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(3000, 4, 110, 120) (3000, 1, 440, 120)
Using 1.18 GB
torch.Size([3000, 1, 440, 120])
torch.Size([3000, 15])


...

Epoch [1/100], Step [60/60], Loss: 162.3417
Epoch [11/100], Step [60/60], Loss: 67.4426
Epoch [21/100], Step [60/60], Loss: 15.3256
Epoch [31/100], Step [60/60], Loss: 5.8447
Epoch [41/100], Step [60/60], Loss: 1.5009
Epoch [51/100], Step [60/60], Loss: 0.4387
Epoch [61/100], Step [60/60], Loss: 0.2114
Epoch [71/100], Step [60/60], Loss: 0.1482
Epoch [81/100], Step [60/60], Loss: 0.1090
Epoch [91/100], Step [60/60], Loss: 0.0841
Finished Training
Commence calculations...
Accuracy: 99.99583350693722 %
Accuracy: 99.98333611064822 %
Results: [0.99995834 0.99983336 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(3000, 4, 110, 120) (3000, 1, 440, 120)
Using 1.18 GB
torch.Size([3000, 1, 440, 120])
torch.Size([3000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [60/60], Loss: 162.5104
Epoch [11/100], Step [60/60], Loss: 103.7294
Epoch [21/100], Step [60/60], Loss: 29.0256
Epoch [31/100], Step [60/60], Loss: 7.7605
Epoch [41/100], Step [60/60], Loss: 2.6867
Epoch [51/100], Step [60/60], Loss: 0.5039
Epoch [61/100], Step [60/60], Loss: 0.3178
Epoch [71/100], Step [60/60], Loss: 0.1688
Epoch [81/100], Step [60/60], Loss: 0.1212
Epoch [91/100], Step [60/60], Loss: 0.0937
Finished Training
Commence calculations...
Accuracy: 99.99583350693722 %
Accuracy: 99.98333611064822 %
Results: [0.99995834 0.99983336 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(3000, 4, 110, 120) (3000, 1, 440, 120)
Using 1.18 GB
torch.Size([3000, 1, 440, 120])
torch.Size([3000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [60/60], Loss: 162.4538
Epoch [11/100], Step [60/60], Loss: 109.2902
Epoch [21/100], Step [60/60], Loss: 34.7730
Epoch [31/100], Step [60/60], Loss: 10.8624
Epoch [41/100], Step [60/60], Loss: 4.7431
Epoch [51/100], Step [60/60], Loss: 1.1891
Epoch [61/100], Step [60/60], Loss: 0.4153
Epoch [71/100], Step [60/60], Loss: 0.1897
Epoch [81/100], Step [60/60], Loss: 0.1392
Epoch [91/100], Step [60/60], Loss: 0.1066
Finished Training
Commence calculations...
Accuracy: 99.99583350693722 %
Accuracy: 99.98333611064822 %
Results: [0.99995834 0.99983336 0.        ]


### different starts

In [4]:
acc=np.zeros((num_trials,3)) #train,test,noisy

for i in range(num_trials):
    data=loaded(t=20,filename="X_data_15.npz")
    data.applySobel()
    data.different_starts(t=4)
    train_loader,test_loader=org_data(data,(len(data.X),1,T_SIZE*data.X.shape[2],data.X.shape[3]))
    model,history=run(train_loader,0,4,num_epochs = EPOCHS)
    print("Commence calculations...")
    acc[i][0]=calc(model,train_loader)
    acc[i][1]=calc(model,test_loader)
    preds=model(test_data_sliced_tensor)
    acc[i][2]=calc_noisy(preds,labels)
    print("Results:",acc[i])
np.save(path+"/data/20_TRIALS_CNN_DIFFERENT_AUGMENTATION",acc)

Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [300/300], Loss: 230.2119
Epoch [11/100], Step [300/300], Loss: 44.6303
Epoch [21/100], Step [300/300], Loss: 7.3972
Epoch [31/100], Step [300/300], Loss: 2.7725
Epoch [41/100], Step [300/300], Loss: 1.1971
Epoch [51/100], Step [300/300], Loss: 0.5400
Epoch [61/100], Step [300/300], Loss: 0.3863
Epoch [71/100], Step [300/300], Loss: 0.1959
Epoch [81/100], Step [300/300], Loss: 0.1334
Epoch [91/100], Step [300/300], Loss: 0.1124
Finished Training
Commence calculations...
Accuracy: 99.99916667361106 %
Accuracy: 99.99666677777408 %
Results: [0.99999167 0.99996667 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [300/300], Loss: 279.8280
Epoch [11/100], Step [300/300], Loss: 48.4789
Epoch [21/100], Step [300/300], Loss: 6.7618
Epoch [31/100], Step [300/300], Loss: 2.3716
Epoch [41/100], Step [300/300], Loss: 1.0184
Epoch [51/100], Step [300/300], Loss: 0.4988
Epoch [61/100], Step [300/300], Loss: 0.2857
Epoch [71/100], Step [300/300], Loss: 0.1807
Epoch [81/100], Step [300/300], Loss: 0.1918
Epoch [91/100], Step [300/300], Loss: 0.0998
Finished Training
Commence calculations...
Accuracy: 99.99916667361106 %
Accuracy: 99.99666677777408 %
Results: [0.99999167 0.99996667 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


...

Epoch [1/100], Step [300/300], Loss: 295.9452
Epoch [11/100], Step [300/300], Loss: 53.1789
Epoch [21/100], Step [300/300], Loss: 7.5437
Epoch [31/100], Step [300/300], Loss: 2.6360
Epoch [41/100], Step [300/300], Loss: 0.8923
Epoch [51/100], Step [300/300], Loss: 0.5881
Epoch [61/100], Step [300/300], Loss: 0.2317
Epoch [71/100], Step [300/300], Loss: 0.1739
Epoch [81/100], Step [300/300], Loss: 0.1380
Epoch [91/100], Step [300/300], Loss: 0.1070
Finished Training
Commence calculations...
Accuracy: 99.99916667361106 %
Accuracy: 99.99666677777408 %
Results: [0.99999167 0.99996667 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [300/300], Loss: 294.3497
Epoch [11/100], Step [300/300], Loss: 67.4623
Epoch [21/100], Step [300/300], Loss: 10.0652
Epoch [31/100], Step [300/300], Loss: 3.5238
Epoch [41/100], Step [300/300], Loss: 1.1094
Epoch [51/100], Step [300/300], Loss: 0.5030
Epoch [61/100], Step [300/300], Loss: 0.4409
Epoch [71/100], Step [300/300], Loss: 0.1951
Epoch [81/100], Step [300/300], Loss: 0.1566
Epoch [91/100], Step [300/300], Loss: 0.1194
Finished Training
Commence calculations...
Accuracy: 99.99916667361106 %
Accuracy: 99.99666677777408 %
Results: [0.99999167 0.99996667 0.        ]
Dataset size: 3000 
Window size: 20 
Image: (110, 120)
Memory needed: 0.74 GB
LOADED DATASET...
Memory left 23.59 GB
(15000, 4, 110, 120) (15000, 1, 440, 120)
Using 5.9 GB
torch.Size([15000, 1, 440, 120])
torch.Size([15000, 15])


/its/home/drs25/RoboSkin/Code/NewRigExperiments/data_format.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.float32).to(device)


Epoch [1/100], Step [300/300], Loss: 259.0715
Epoch [11/100], Step [300/300], Loss: 50.4615


### all augmentation

In [ ]:
acc=np.zeros((num_trials,3)) #train,test,noisy

for i in range(num_trials):
    data=loaded(t=20,filename="X_data_15.npz")
    data.applySobel()
    data.speed_augment(t=4)
    data.zoom_augment([10,20,30,40])
    data.orientation_augment()
    train_loader,test_loader=org_data(data,(len(data.X),1,T_SIZE*data.X.shape[2],data.X.shape[3]))
    model,history=run(train_loader,0,4,num_epochs = EPOCHS)
    print("Commence calculations...")
    acc[i][0]=calc(model,train_loader)
    acc[i][1]=calc(model,test_loader)
    preds=model(test_data_sliced_tensor)
    acc[i][2]=calc_noisy(preds,labels)
    print("Results:",acc[i])
np.save(path+"/data/20_TRIALS_CNN_ALL_AUGMENTATION",acc)

## viewer

In [ ]:
data=loaded(t=20,filename="X_data_15.npz")
data.applySobel()
example_vid=data.X[0]

#example_speed=

In [19]:
import pandas as pd
orient=np.load(path+"/data/20_TRIALS_CNN_ORIENTATION_AUGMENTATION.npy")
speeds=np.load(path+"/data/20_TRIALS_CNN_SPEED_AUGMENTATION.npy")
zoom=np.load(path+"/data/20_TRIALS_CNN_ZOOM_AUGMENTATION.npy")
none=np.load(path+"/data/20_TRIALS_CNN_NO_AUGMENTATION.npy")
diff=np.load(path+"/data/20_TRIALS_CNN_DIFFERENT_AUGMENTATION.npy")
#all_=np.load(path+"/data/20_TRIALS_CNN_ALL_AUGMENTATION.npy")

df = pd.DataFrame(
    np.vstack([orient, speeds, zoom, none, diff,]),  # Stack all datasets
    columns=["Train", "Test", "New"]
)
# Add a column indicating the dataset source
df["Dataset"] = np.repeat(["Orientation", "Speed", "Zoom","None","All"], 20)

# Get summary statistics for each dataset separately
summary = df.groupby("Dataset")[["Train", "Test", "New"]].agg(["mean", "std", "min", "max"])

# Display summary
print(summary)

                Train                               Test                      \
                 mean  std       min       max      mean       std       min   
Dataset                                                                        
None         0.999958  0.0  0.999958  0.999958  0.999833  0.000000  0.999833   
Orientation  0.999986  0.0  0.999986  0.999986  0.999778  0.000261  0.999389   
Speed        0.999986  0.0  0.999986  0.999986  0.999944  0.000000  0.999944   
Zoom         0.999990  0.0  0.999990  0.999990  0.999958  0.000000  0.999958   

                       New                 
                  max mean  std  min  max  
Dataset                                    
None         0.999833  0.0  0.0  0.0  0.0  
Orientation  0.999944  0.0  0.0  0.0  0.0  
Speed        0.999944  0.0  0.0  0.0  0.0  
Zoom         0.999958  0.0  0.0  0.0  0.0  


## stats testing

In [20]:
from scipy.stats import ranksums

datasets = {
    "orient": orient,
    "speeds": speeds,
    "zoom": zoom,
    "none": none,
    "all" : all_
}

# Compare datasets pairwise
pairs = [
    ("orient", "speeds"),
    ("orient", "zoom"),
    ("orient", "none"),
    ("orient", "all"),
    ("speeds", "zoom"),
    ("speeds", "none"),
    ("speeds", "all"),
    ("zoom", "none"),
    ("zoom", "all"),
    ("none", "all")
]
alpha = 0.05

for i, col in enumerate(["Train", "Test", "New"]):
    print(f"\n=== Rank-Sum Test for {col} Accuracy ===")
    for d1, d2 in pairs:
        stat, p_value = ranksums(datasets[d1][:, i], datasets[d2][:, i])  # Compare column i
        significance = "Statistically Significant" if p_value < alpha else "Not Significant"
        print(f"{d1} vs {d2}: stat={stat:.3f}, p-value={p_value:.3f} → {significance}")


=== Rank-Sum Test for Train Accuracy ===
orient vs speeds: stat=0.000, p-value=1.000 → Not Significant
orient vs zoom: stat=-5.410, p-value=0.000 → Statistically Significant
orient vs none: stat=5.410, p-value=0.000 → Statistically Significant
speeds vs zoom: stat=-5.410, p-value=0.000 → Statistically Significant
speeds vs none: stat=5.410, p-value=0.000 → Statistically Significant
zoom vs none: stat=5.410, p-value=0.000 → Statistically Significant

=== Rank-Sum Test for Test Accuracy ===
orient vs speeds: stat=-1.623, p-value=0.105 → Not Significant
orient vs zoom: stat=-5.410, p-value=0.000 → Statistically Significant
orient vs none: stat=2.164, p-value=0.030 → Statistically Significant
speeds vs zoom: stat=-5.410, p-value=0.000 → Statistically Significant
speeds vs none: stat=5.410, p-value=0.000 → Statistically Significant
zoom vs none: stat=5.410, p-value=0.000 → Statistically Significant

=== Rank-Sum Test for New Accuracy ===
orient vs speeds: stat=0.000, p-value=1.000 → Not Si